# Passo a Passo de Implementação do Projeto

## 0) Visão geral do que já existe

- **Treino**: `train.py` treina um `RandomForest` dentro de um `Pipeline` com `StandardScaler`, calcula métricas, salva artefatos em `artifacts/model` e (se disponível) registra no MLflow.  
- **Config**: `configs.yaml` define `test_size`, `random_state`, hiperparâmetros e pasta de export.  
- **Dados**: `data_prep.py` usa `load_breast_cancer` do scikit-learn e faz split estratificado.  
- **Util**: `utils.py` carrega YAML (a função `load_config`).  
- **Tracking**: `docker-compose.mlflow.yml` sobe um MLflow Server file-based na porta 5000.  
- **Kubernetes**: `deployment.yaml` define Deployment/Service para a imagem `projeto10-model:latest` (porta 8000, `/ping`, NodePort 30080).  

---

## 1) Preparar ambiente local

```bash
python -m venv .venv
source .venv/bin/activate        # Windows: .venv\Scripts\activate
python -m pip install -U pip

# instalar dependências do projeto
pip install -r requirements.txt

# (opcional) MLflow p/ tracking/serving
pip install mlflow
```

> O script principal chama `load_config('src/mlops_project/configs.yaml')` e usa imports relativos (`from .utils ...`), então vamos executar como **módulo** a partir de `src/`.

---

## 2) Conferir e ajustar `configs.yaml`

Abra `src/mlops_project/configs.yaml` e ajuste conforme sua necessidade (já vem com valores padrão):  
- `test_size`, `random_state`  
- `model.params` (ex.: `n_estimators`, `max_depth`, `n_jobs`)  
- `paths.exported_model_dir` (ex.: `artifacts/model`)  

---

## 3) Executar o treino e gerar artefatos

O dataset vem de `load_breast_cancer(as_frame=True)` e o split é estratificado.  

Da raiz do repositório:

```bash
cd projeto10-mlops-slim-mlflow-fastapi-optional/core/src
python -m mlops_project.train

```

Fluxo de execução:
- Carrega config (`utils.load_config`) e dados (`data_prep.load_dataset`).  
- Split treino/teste com `test_size` e `random_state` do YAML.  
- Monta `Pipeline([('scaler', StandardScaler(with_mean=False)), ('clf', RandomForestClassifier(**params))])`, treina, prediz e calcula **accuracy**, **precision**, **recall**, **f1**, imprimindo também o `classification_report`.  
- Cria pasta `artifacts/model` (ou a do YAML) e salva `model.joblib`.  
- Se o MLflow estiver instalado/importado, salva um **MLflow Model** no mesmo diretório e registra **parâmetros/métricas/model** no servidor (caso `MLFLOW_TRACKING_URI` esteja configurado).  

---

## 4) Subir o MLflow Tracking

Vídeo como instalar Docker e Docker compose:

https://www.youtube.com/watch?v=05YN8F8ajBc&t=10s  (no windows)

https://www.youtube.com/watch?v=h27ZVQIh7Ro (docker compose no Linux)

https://www.youtube.com/watch?v=Gpal5KsSHMQ&t=1s (docker no Linux)

Suba o servidor com o compose que você já tem:

```bash
cd projeto10-mlops-slim-mlflow-fastapi-optional/core                                               
docker compose -f docker-compose.yml up -d                                                                                      
 
 #UI em http://localhost:5000
```

No terminal onde você vai treinar, aponte o client para o servidor e (opcional) defina o nome do experimento:

```bash
export MLFLOW_TRACKING_URI=http://127.0.0.1:5000
export MLFLOW_EXPERIMENT_NAME=Projeto10_MLOps
```

> Reexecute o **passo 3** para registrar runs, parâmetros, métricas e o modelo no MLflow. O `train.py` já está preparado para isso.

---

## 5) Servir o modelo localmente (MLflow Models Serve)

Como o `train.py` salva um **MLflow Model** dentro da pasta de export, você pode servir direto dela:

```bash
mlflow models serve   -m artifacts/model   -p 8000 --host 0.0.0.0 --no-conda
```

### Testes rápidos

```bash
# healthcheck (compatível com /ping do seu manifest)
curl -s http://127.0.0.1:8000/ping

# inferência
curl -X POST http://127.0.0.1:8000/invocations \
  -H "Content-Type: application/json" \
  -d '{
    "dataframe_split": {
      "columns": [
        "mean radius","mean texture","mean perimeter","mean area","mean smoothness",
        "mean compactness","mean concavity","mean concave points","mean symmetry","mean fractal dimension",
        "radius error","texture error","perimeter error","area error","smoothness error",
        "compactness error","concavity error","concave points error","symmetry error","fractal dimension error",
        "worst radius","worst texture","worst perimeter","worst area","worst smoothness",
        "worst compactness","worst concavity","worst concave points","worst symmetry","worst fractal dimension"
      ],
      "data": [[
        14.0,20.0,90.0,600.0,0.10,
        0.15,0.12,0.07,0.18,0.06,
        0.4,1.2,3.0,40.0,0.01,
        0.02,0.03,0.02,0.03,0.004,
        16.5,27.0,110.0,850.0,0.13,
        0.25,0.22,0.12,0.24,0.08
      ]]
    }
  }'
'
```

> Dica: se preferir, salve o MLflow Model em um **subdiretório** (ex.: `artifacts/model/mlflow_model`) para não misturar com o `model.joblib`.

---

## 6) Empacotar a aplicação em Docker

A imagem esperada pelo seu `Deployment` do K8s se chama **`projeto10-model:latest`** e expõe a **porta 8000** com um endpoint **`/ping`**.

Construa a imagem com seu `Dockerfile`:

```bash
docker build -t projeto10-model:latest .
```

Teste localmente:

```bash
docker run --rm -p 8000:8000 projeto10-model:latest
curl -s http://127.0.0.1:8000/ping
```

---

## 7) Publicar no Kubernetes

Aplique os manifests que você já possui:


https://www.youtube.com/watch?v=mi_aotXDMR8 (como instalar Kubernetes - kubectl)

```bash
kubectl apply -f projeto10-mlops-slim-mlflow-fastapi-optional/core/k8s/deployment.yaml
kubectl get pods
kubectl get svc
kubectl get deploy projeto10-model
kubectl rollout status deploy/projeto10-model
kubectl get pods -l app=projeto10-model -o wide
kubectl logs -l app=projeto10-model --tail=100

```

- **Deployment**: 2 réplicas, `image: projeto10-model:latest`, probes em `/ping` na porta 8000.  
- **Service**: tipo **NodePort**, mapeando `:80 -> 8000` e expondo **30080** no nó.  

### Testes

```bash
# se for NodePort e você está no mesmo nó (minikube/kind):
curl -s http://127.0.0.1:30080/ping

# ou faça port-forward
kubectl port-forward svc/projeto10-model-svc 8080:80
curl -s http://127.0.0.1:8080/ping

# inferência (mesmo payload do passo 5)
curl -X POST http://127.0.0.1:8080/invocations   -H "Content-Type: application/json"   -d '{"dataframe_split":{"columns":["mean radius","mean texture","mean perimeter"],"data":[[14.0,20.0,90.0]]}}'
```

---

